#環境構築

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow==1.15

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 48.7 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 3.8 MB 34.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6c694071643547afdedeb7b9cdeed1894bfda9fdfcafbb4f8aa750d322a67aca
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard

In [3]:
import numpy
numpy.__version__

'1.19.5'

In [4]:
%cd /content/drive/MyDrive/後期/きゅうり1/TF/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/MyDrive/後期/きゅうり1/TF/models/research


In [5]:
#確認用（実行しなくてもいい）
%cd /content/drive/MyDrive/後期/きゅうり1/TF/models/research/
%env PYTHONPATH=/env/python:/content/drive/MyDrive/後期/きゅうり1/TF/models/research/:/content/drive/MyDrive/後期/きゅうり1/TF/models/research/slim
%env
!python object_detection/builders/model_builder_test.py

/content/drive/MyDrive/後期/きゅうり1/TF/models/research
env: PYTHONPATH=/env/python:/content/drive/MyDrive/後期/きゅうり1/TF/models/research/:/content/drive/MyDrive/後期/きゅうり1/TF/models/research/slim
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_mat

環境構築（スプレッドシートに出力する場合）

In [6]:
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

#画像判別
参考：https://github.com/karaage0703/object_detection_tools/blob/master/scripts/object_detection.py

In [7]:
import cv2
import numpy as np
import os
import sys
import time
import tensorflow as tf
from google.colab.patches import cv2_imshow

from distutils.version import StrictVersion

if StrictVersion(tf.__version__) < StrictVersion('1.14.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.14.*.')

In [473]:
model = '/content/drive/MyDrive/後期/きゅうり1/保存/infarence_graph/frozen_inference_graph.pb'
input_file = '/content/drive/MyDrive/後期/きゅうり1/キュウリ検出用/test50.jpg'

detection_graph = tf.Graph()

mode = 'bbox'

colors = [
  (0, 0, 0),
  (0, 75, 255),
  (0, 255, 255),
  (255, 75, 0)
]

In [474]:
def load_graph():
  with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(model, 'rb') as fid:
      serialized_graph = fid.read()
      od_graph_def.ParseFromString(serialized_graph)
      tf.import_graph_def(od_graph_def, name='')
    return detection_graph

In [475]:
def mosaic(src, ratio=0.1):
    small = cv2.resize(src, None, fx=ratio, fy=ratio, interpolation=cv2.INTER_NEAREST)
    return cv2.resize(small, src.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

In [476]:
def mosaic_area(src, x_min, y_min, x_max, y_max, ratio=0.1):
    dst = src.copy()
    dst[y_min:y_max, x_min:x_max] = mosaic(dst[y_min:y_max, x_min:x_max], ratio)
    return dst

In [477]:
print('Loading graph...')
detection_graph = load_graph()
print('Graph is loaded')

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

with detection_graph.as_default():
  tf_sess = tf.Session(config = tf_config)
  ops = tf.get_default_graph().get_operations()
  all_tensor_names = {output.name for op in ops for output in op.outputs}
  tensor_dict = {}
  for key in [
      'num_detections', 'detection_boxes', 'detection_scores',
      'detection_classes', 'detection_masks'
  ]:
    tensor_name = key + ':0'
    if tensor_name in all_tensor_names:
      tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
          tensor_name)

  image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

Loading graph...
Graph is loaded


In [478]:
def run_inference_for_single_image(image, graph):
  # Run inference
  output_dict = tf_sess.run(tensor_dict,
                          feed_dict={image_tensor: image})

  # all outputs are float32 numpy arrays, so convert types as appropriate
  output_dict['num_detections'] = int(output_dict['num_detections'][0])
  output_dict['detection_classes'] = output_dict[
      'detection_classes'][0].astype(np.int64)
  output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
  output_dict['detection_scores'] = output_dict['detection_scores'][0]
  return output_dict

In [479]:
labels = ['background','cucumber']
#labels = ['background' ,'raccoon' ,'civet' ,'cat']
#labels = ['background' ,'cat' ,'raccoon' ,'civet']

img = cv2.imread(input_file)

img_bgr = cv2.resize(img, (300, 300))

# convert bgr to rgb
image_np = img_bgr[:,:,::-1]
image_np_expanded = np.expand_dims(image_np, axis=0)
output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)

for i in range(output_dict['num_detections']):
  class_id = output_dict['detection_classes'][i]
  if class_id < len(labels):
    label = labels[class_id]
  else:
    label = 'unknown'

  detection_score = output_dict['detection_scores'][i]

  if detection_score > 0.5:
      # Define bounding box
      h, w, c = img.shape
      box = output_dict['detection_boxes'][i] * np.array( \
        [h, w,  h, w])
      box = box.astype(np.int)

      if mode == 'bbox':
        color = colors[class_id] if class_id < len(colors) else colors[len(colors)-1]

        # Draw bounding box
        cv2.rectangle(img, \
          (box[1], box[0]), (box[3], box[2]), color, 3)

        # Put label near bounding box
        information = '%s: %.1f%%' % (label, output_dict['detection_scores'][i] * 100.0)
        cv2.rectangle(img, \
          (box[1], box[2] - 15), (box[1] + (int(len(information))*10), box[2]), color, -1)
        cv2.putText(img, information, (box[1] + 5, box[2] - 2), \
          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
      elif mode == 'mosaic':
        img = mosaic_area(img, box[1], box[0], box[3], box[2], ratio=0.05)

cv2_imshow(img)

tf_sess.close()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

# 動画検出
環境構築までのコードを実行した後に、下のコードを実行することで、動画での物体検出が可能になります。以下は手順の説明です。

ob_fps_.pyをMy Driveにアップロードする。変更点は12行目のlabelsのみ。

-mの引数に学習させたモデルのfrozen_inference_graph.pbファイルを指定（パスはMy Drive以降のみ）

-iの引数に検出させたい動画ファイルを指定（パスはMy Drive以降のみ）

停止後にoutput.mp4ファイルがMy Driveに保存されます

ーーー2020/11/16；mp4ファイルが作成されないエラーがありました。

その場合はob_fps_.pyの73行目「MP4V→DIV3」
74行目「output.mp4→output.avi」に変更で解決します。

ーーー2020/11/22；学習フォルダにob_fps_output.pyを追加しました。物体検出確認用.ipynbの環境構築欄にスプレッドシード出力する場合のコードを追加しました。

ob_fps_output.pyの変更点はlabelsのみ。

In [480]:
%cd /content/drive/My\ Drive
!python3 ob_fps_.py -m '/test/infarence_graph/frozen_inference_graph.pb' -i 'movie/movie.mp4'

/content/drive/My Drive
python3: can't open file 'ob_fps_.py': [Errno 2] No such file or directory


In [481]:
#スプレッドシート出力させる場合はこちらを実行
%cd /content/drive/My\ Drive
!python3 ob_fps_output.py -m '/test/infarence_graph/frozen_inference_graph.pb' -i 'movie/movie.mp4'

/content/drive/My Drive
python3: can't open file 'ob_fps_output.py': [Errno 2] No such file or directory
